In [27]:
# !pip install python-dotenv
!pip install openai

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access the API_KEY environment variable
OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [3]:
import requests


def get_current_weather(city="London", api_key=OPENWEATHER_API_KEY):
    # Define the base URL for the OpenWeather API
    base_url = "http://api.openweathermap.org/data/2.5/weather"

    # Construct the full URL with query parameters
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric"  # Use metric units (Celsius)
    }

    # Make the GET request to the API
    response = requests.get(base_url, params=params)

    return response


In [4]:
response = get_current_weather(city="Kyiv", api_key=OPENWEATHER_API_KEY)

In [5]:
response.status_code

200

In [6]:
response.json()

{'coord': {'lon': 30.5167, 'lat': 50.4333},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03n'}],
 'base': 'stations',
 'main': {'temp': 14.29,
  'feels_like': 13.84,
  'temp_min': 14.25,
  'temp_max': 14.29,
  'pressure': 1012,
  'humidity': 79,
  'sea_level': 1012,
  'grnd_level': 996},
 'visibility': 10000,
 'wind': {'speed': 0.89, 'deg': 139, 'gust': 2.24},
 'clouds': {'all': 50},
 'dt': 1727810765,
 'sys': {'type': 2,
  'id': 2003742,
  'country': 'UA',
  'sunrise': 1727755099,
  'sunset': 1727796994},
 'timezone': 10800,
 'id': 703448,
 'name': 'Kyiv',
 'cod': 200}

In [7]:
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    weather_data = response.json()

    # Extract useful information
    temperature = weather_data['main']['temp']
    weather_description = weather_data['weather'][0]['description']
    city_name = weather_data['name']

    # Print the result
    print(f"The current temperature in {city_name} is {temperature}°C.")
    print(f"Weather description: {weather_description.capitalize()}")
else:
    # Print the error if the request was not successful
    print(f"Failed to retrieve data: {response.status_code}")
    print(response.json())  # Display error message from API

The current temperature in Kyiv is 14.29°C.
Weather description: Scattered clouds


In [8]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)


def generate_openai_response(weather_data):
    # Extract useful information
    temperature = weather_data['main']['temp']
    weather_description = weather_data['weather'][0]['description']
    city_name = weather_data['name']

    # Prepare a prompt for the OpenAI API
    # prompt = (f"The weather in {city_name} is currently {weather_description} with a temperature of {temperature}°C. "
    #           f"Please generate a friendly message about the weather in a conversational tone. ")

    prompt = f"The weather in {city} is currently {weather_description} with a temperature of {temperature}°C. Now, imagine you're talking to a group of Gen-Z students who are deciding how to spend their day. Your goal is to encourage them to use this weather as an excuse to dive into their Python for Data Science labs. Make it fun, relatable, and hype them up with the kind of language Gen-Z would use. Keep it cool, motivating, and relevant to coding – like turning a chill or rainy day into the perfect excuse to learn, grind, and level up their Python skills. Make it short. Translate it to Ukrainian and return only Ukrainian message."
    
    # Make a request to the OpenAI ChatCompletion API
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a friendly and conversational assistant."},
            {"role": "user", "content": prompt}
        ], temperature=1,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={"type": "text"
                         }
    )

    return response


In [9]:
# Ask the user for the city
city = input("Enter a city to get the weather: ")

try:
    # Fetch weather data
    response = get_current_weather(city=city, api_key=OPENWEATHER_API_KEY)

    weather_data = response.json()

    # Generate a friendly response using OpenAI
    openai_response = generate_openai_response(weather_data)

    print("\n--- Weather Information ---")
    print(f"City: {weather_data['name']}")
    print(f"Temperature: {weather_data['main']['temp']}°C")
    print(f"Weather: {weather_data['weather'][0]['description'].capitalize()}")

    print("\n--- Chatbot Response ---")
    print(openai_response.choices[0].message.content)

except Exception as e:
    print(f"An error occurred: {e}")

Enter a city to get the weather: Kyiv
An error occurred: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************MMit. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


In [40]:
# Get the generated text
print(openai_response.choices[0].message.content)

Ей, команда! Погода в Києві супер – хмарки на небі, температура майже 30°C. Це не час просто байдикувати! Давайте переведемо цю chill атмосферу в енергію для нашого Python for Data Science. Уявіть, наскільки крутими ми станемо, коли зможемо з легкістю шарити в даних і робити круті графіки. Завантажуйте ноутбуки, розгортуйте Jupyter і вперед, кодити й прокачувати свої скіли! Погода може бути як завгодно – а наше майбутнє в наших руках. Полетіли разом до вершини! 🚀📈👩‍💻👨‍💻


In [45]:
openai_response.choices[0].message.content

'Ей, команда! Погода в Києві супер – хмарки на небі, температура майже 30°C. Це не час просто байдикувати! Давайте переведемо цю chill атмосферу в енергію для нашого Python for Data Science. Уявіть, наскільки крутими ми станемо, коли зможемо з легкістю шарити в даних і робити круті графіки. Завантажуйте ноутбуки, розгортуйте Jupyter і вперед, кодити й прокачувати свої скіли! Погода може бути як завгодно – а наше майбутнє в наших руках. Полетіли разом до вершини! 🚀📈👩\u200d💻👨\u200d💻'

In [50]:
openai_response

ChatCompletion(id='chatcmpl-ACRRUT6kGzBRE1fW8JgbA60vjqKUK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Ей, команда! Погода в Києві супер – хмарки на небі, температура майже 30°C. Це не час просто байдикувати! Давайте переведемо цю chill атмосферу в енергію для нашого Python for Data Science. Уявіть, наскільки крутими ми станемо, коли зможемо з легкістю шарити в даних і робити круті графіки. Завантажуйте ноутбуки, розгортуйте Jupyter і вперед, кодити й прокачувати свої скіли! Погода може бути як завгодно – а наше майбутнє в наших руках. Полетіли разом до вершини! 🚀📈👩\u200d💻👨\u200d💻', role='assistant', function_call=None, tool_calls=None, refusal=None))], created=1727528788, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_c17d3befe7', usage=CompletionUsage(completion_tokens=166, prompt_tokens=149, total_tokens=315, completion_tokens_details={'reasoning_tokens': 0}))

In [71]:
# # Generate a friendly response using OpenAI
# openai_response = generate_openai_response(weather_data)